# Traffic Light Identification and Classification
Senior Project
Team PineApple

# Setup

## Importing Packages

In [ ]:
import time
print("Time loaded")
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
print("Pandas loaded")
import numpy as np # linear algebra
print("Numpy loaded")
import yaml # read yaml file
print("Yaml loaded")
import os # For file path
print("Os loaded")
import cv2 # For image processing
print("OpenCV loaded")
import sklearn as sk # For machine learning
print("Sklearn loaded")

import matplotlib.pyplot as plt
print("Matplotlib loaded")
import tensorflow as tf
print("Tensorflow loaded")
from shutil import copyfile
print("Shutil loaded")

import torch # For neural network
print("Torch loaded")
import PIL
print("PIL loaded")

## Data Pre-Processing

Data available at https://hci.iwr.uni-heidelberg.de/content/bosch-small-traffic-lights-dataset

In [ ]:
train_file_path = 'train.yaml'

In [ ]:
train_key = yaml.load(open(train_file_path), Loader=yaml.FullLoader)

In [ ]:
def annotatePicture(dictEntry):
    img = cv2.imread(dictEntry['path']).copy()
    for i in dictEntry['boxes']: 
        startPoint = (int(i['x_min']), int(i['y_min']))
        endPoint = (int(i['x_max']), int(i['y_max']))
        cv2.rectangle(img, startPoint, endPoint, (0,255,0), 2)
    return img

In [ ]:
# Used to show test output
img = annotatePicture(train_key[23])
plt.imshow(img)

# Model Time

In [ ]:
# Displaying encoding labels
labels = []
for i in range(len(train_key)):
    for boxes in train_key[i]['boxes']:
                if(boxes['label'] not in labels):
                    labels.append(boxes['label'])
labelDict = {}
for i in labels:
    labelDict[i] = labels.index(i)

In [ ]:
def makeAnnotations(train_key, labelDict):
    # For all images
    # for i in range(len(train_key)):
    for i in range(30):
        filePath = train_key[i]['path']
        indexOfName = filePath.rfind('/') + 1 # Get index of last '/'
        annotationPath = filePath[indexOfName:].replace(".png", ".txt") # Get shortened name of file
        copyfile(filePath, "./dataset/train/" + annotationPath.replace(".txt", ".png")) # Copy img to new location
        
        f = open("./dataset/train/" + annotationPath, 'w') # .txt annotation file to make
        f.truncate(0)
        img_height, img_width = cv2.imread(filePath).shape[:2]
        # For all boxes
        for boxes in train_key[i]['boxes']:
            xmin = int(boxes['x_min'])
            ymin = int(boxes['y_min'])
            xmax = int(boxes['x_max'])
            ymax = int(boxes['y_max'])
            avg_x = (xmin + xmax) / 2
            avg_y = (ymin + ymax) / 2
            yolo_width = (xmax - avg_x) / img_width
            yolo_height = (ymax - avg_y) / img_height
            yolo_x = avg_x / img_width
            yolo_y = 1 - (avg_y / img_height)
            yolo_label = labelDict[boxes['label']]
            f.write(str(yolo_label) + " " + str(yolo_x) + " " + str(yolo_y) + " " + str(yolo_width) + " " + str(yolo_height) + "\n")
        f.close()

In [ ]:
makeAnnotations(train_key, labelDict)

In [ ]:
def makeConfig(path, train, val, test, labelDict):
    f = open("config.yaml", 'w')
    f.truncate(0)
    path = "dataset/"
    f.write("path: " + path + "\n")
    f.write("train: " + train + "/\n")
    f.write("val: " + val + "/\n")
    # f.write("train: " + test + "/\n")
    f.write("\n")
    f.write("nc: " + str(len(labelDict)) + "\n")
    f.write("\n")
    f.write("names: " + str(labelDict.keys())[10:-1] + "\n")
    f.close()

In [ ]:
makeConfig("../dataset/", "train", "train", "test", labelDict) #train and val are same folder for now

In [ ]:
if not (os.path.isdir("yolov5")):
    os.popen("./scripts/importYolo.sh")

In [ ]:
imgs = 720
batch = 16
epochs = 12
configFile = "config.yaml"
model = "yolov5s.pt"
name = "yolo_light_detection"

!python yolov5/train.py --img $imgs --batch $batch --epoch $epochs --data $configFile --weights $model --name $name

In [ ]:
!python yolov5/detect.py --source ./dataset/train/ --weights yolov5/runs/train/yolo_light_detection5/weights/best.pt --conf 0.25 --name yolo_light_detection_test